In [ ]:
# For interactive matplotlib
%matplotlib widget

# Import the N_Queens_Game class from the mobile_game module
from mobile_game import N_Queens_Game

# Create an instance of N_Queens_Game
n_queens_game = N_Queens_Game()

# Setup the game interface
n_queens_game.setup()

Output()